## Exploring Preprocessed fMRI Data from fMRIPREP

BIDS applications such as fMRIPREP output data into a full data structure with strong similarity to BIDS organization principals. In fact, there is a specification for derivatives (outputs derived from) BIDS datasets; although this is a current work in progress, details can be found in: [BIDS Derivatives](https://bids-specification.readthedocs.io/en/latest/06-extensions.html). 

In this tutorial, we'll explore the outputs generated by fMRIPREP and get a handle of how the data is organized from this preprocessing pipeline
***

Luckily the semi-standardized output for fMRIPREP is organized in such a way that the data is easily accessible using PyBIDS! We'll first show what the full data-structure looks like then provide you methods on how you can pull specific types of outputs using PyBIDS.

### The fMRIPREP Derivative Data Structure

First let's take a quick look at the fmriprep data structure:

In [1]:
!tree -L 1 '../data/ds000030/derivatives/fmriprep/'

../data/ds000030/derivatives/fmriprep/
├── dataset_description.json
├── sub-10438
├── sub-10788
├── sub-11108
└── sub-50052

4 directories, 1 file


First note that inside the fmriprep folder, we have a folder per-subject. Let's take a quick look at a single subject folder:

In [2]:
!tree '../data/ds000030/derivatives/fmriprep/sub-10788/'

../data/ds000030/derivatives/fmriprep/sub-10788/
├── anat
│   ├── sub-10788_desc-aparcaseg_dseg.nii.gz
│   ├── sub-10788_desc-aseg_dseg.nii.gz
│   ├── sub-10788_desc-brain_mask.json
│   ├── sub-10788_desc-brain_mask.nii.gz
│   ├── sub-10788_desc-preproc_T1w.json
│   ├── sub-10788_desc-preproc_T1w.nii.gz
│   ├── sub-10788_dseg.nii.gz
│   ├── sub-10788_from-fsnative_to-T1w_mode-image_xfm.txt
│   ├── sub-10788_from-T1w_to-fsnative_mode-image_xfm.txt
│   ├── sub-10788_label-CSF_probseg.nii.gz
│   ├── sub-10788_label-GM_probseg.nii.gz
│   ├── sub-10788_label-WM_probseg.nii.gz
│   ├── sub-10788_space-MNI152NLin2009cAsym_desc-brain_mask.json
│   ├── sub-10788_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz
│   ├── sub-10788_space-MNI152NLin2009cAsym_desc-preproc_T1w.json
│   ├── sub-10788_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz
│   ├── sub-10788_space-MNI152NLin2009cAsym_dseg.nii.gz
│   ├── sub-10788_space-MNI152NLin2009cAsym_label-CSF_probseg.nii.gz
│   ├── sub-10788_space-MNI152N

As you can see above, each subject folder is organized into an <code>anat</code> and <code>func</code> sub-folder. 

Specifically:

- the <code>anat</code> folder contains the preprocessed anatomical data. If multiple T1 files are available (all T1s even across sessions), then these data are merged - you will always have one <code>anat</code> folder under the subject folder
- the <code>func</code> folder contains the preprocessed functional data. All tasks are dumped into the same folder and like the BIDS convention are indicated by the use of their filenames (<code>task-[task_here]</code>)

***
This data is single-session, so a session folder is missing here - but with multiple sessions you will see <code>anat</code> and <code>ses-[insert_session_here]</code> folders where each session folder contain a <code>func</code> folder.
***

Hopefully you're now convinced that the outputs of fMRIPREP roughly follows BIDS organization principles and is, in fact, quite simple. The filenames themselves give you a full description of what each file is (check the [slides](https://docs.google.com/presentation/d/1er6dQcERL-Yeb5-7A29tJnmqgHNaLpTLXM3e-SmpjDg/edit?usp=sharing) to get an idea of what each file means!

Now let's see how we can pull data in using PyBIDS!

Let's import PyBIDS through the <code>bids</code> module first:

In [3]:
import bids

We can make a <code>bids.BIDSLayout</code> object as usual by just feeding in the fmriprep directory! However, there is one caveat... note that fMRIPrep doesn't exactly adhere to the *standard BIDS convention*. It uses fields such as <code>desc-</code> which are not part of the original BIDS specification. I.e: `sub-10788_desc-preproc_T1w.nii.gz`

In fact, BIDS allows for *extensions* which enable you to add additional fields to the standard BIDS convention (such as <code>desc-</code>!). fMRIprep uses the <code>derivatives</code> extension of the BIDS standard. PyBIDS can handle standard extensions to the BIDS specification quite easily:


In [4]:
layout = bids.BIDSLayout('../data/ds000030/derivatives/fmriprep/',
                        config=['bids', 'derivatives'])

/home/jerry/.pyenv/versions/3.7.0/envs/sdc-bids-fmri/lib/python3.7/site-packages/bids/layout/models.py:102: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


Now that we have a layout object, we can pretend like we're working with a BIDS dataset! Let's try some common commands that you would've used with a BIDS dataset:

**Getting Subjects**

First, we'll demonstrate that we can grab a list of pre-processed subjects much like in the way we would grab subjects from a raw BIDS dataset:


In [5]:
layout.get_subjects()

['10438', '10788', '11108', '50052']

**Getting Tasks**

 We can also do the same for tasks

In [6]:
layout.get_tasks()

['rest']

Now let's try fetching specific files. Similar to how you would fetch BIDS data using PyBIDS, the exact same syntax will work for fMRIPREP derivatives. Let's try pulling just the preprocessed anatomical data. 

Recall that the anatomical folder is organized as follows:

In [7]:
!tree '../data/ds000030/derivatives/fmriprep/sub-10788/anat'

../data/ds000030/derivatives/fmriprep/sub-10788/anat
├── sub-10788_desc-aparcaseg_dseg.nii.gz
├── sub-10788_desc-aseg_dseg.nii.gz
├── sub-10788_desc-brain_mask.json
├── sub-10788_desc-brain_mask.nii.gz
├── sub-10788_desc-preproc_T1w.json
├── sub-10788_desc-preproc_T1w.nii.gz
├── sub-10788_dseg.nii.gz
├── sub-10788_from-fsnative_to-T1w_mode-image_xfm.txt
├── sub-10788_from-T1w_to-fsnative_mode-image_xfm.txt
├── sub-10788_label-CSF_probseg.nii.gz
├── sub-10788_label-GM_probseg.nii.gz
├── sub-10788_label-WM_probseg.nii.gz
├── sub-10788_space-MNI152NLin2009cAsym_desc-brain_mask.json
├── sub-10788_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz
├── sub-10788_space-MNI152NLin2009cAsym_desc-preproc_T1w.json
├── sub-10788_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz
├── sub-10788_space-MNI152NLin2009cAsym_dseg.nii.gz
├── sub-10788_space-MNI152NLin2009cAsym_label-CSF_probseg.nii.gz
├── sub-10788_space-MNI152NLin2009cAsym_label-GM_probseg.nii.gz
└── sub-10788_space-MNI152NLin2009cAsym_labe

The file that we're interested in is of form <code>sub-[subject]_desc-preproc_T1w.nii.gz</code>. Now we can construct a PyBIDS call to pull these types of files specifically:

In [8]:
preproc_T1 = layout.get(datatype='anat', desc='preproc', extension=".nii.gz")
preproc_T1

[<BIDSImageFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-10438/anat/sub-10438_desc-preproc_T1w.nii.gz'>,
 <BIDSImageFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-10438/anat/sub-10438_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz'>,
 <BIDSImageFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-10788/anat/sub-10788_desc-preproc_T1w.nii.gz'>,
 <BIDSImageFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-10788/anat/sub-10788_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz'>,
 <BIDSImageFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-11108/anat/sub-11108_desc-preproc_T1w.nii.gz'>,
 <BIDSImageFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-11108/anat/sub-11108_space-MNI152NLin2009cAsym_d

***
**NOTE**: If we didn't feed in <code>config=['bids','derivatives']</code> to PyBIDS the <code>desc</code> keyword would not work!
***

Note that we also pulled in MNI152NLin2009cAsym_preproc.nii.gz data as well. This is data that has been transformed into MNI152NLin2009cAsym template space. We can pull this data out by further specifying our <code>layout.get</code> using the <code>space</code> argument:

In [9]:
mni_preproc_T1 = layout.get(datatype='anat',desc='preproc',extension='.nii.gz',space='MNI152NLin2009cAsym')
mni_preproc_T1

[<BIDSImageFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-10438/anat/sub-10438_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz'>,
 <BIDSImageFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-10788/anat/sub-10788_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz'>,
 <BIDSImageFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-11108/anat/sub-11108_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz'>,
 <BIDSImageFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-50052/anat/sub-50052_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz'>]

What if we wanted to pull out the data in T1 "native space" (it really is a template space, since it is merged T1s)? Unfortunately for this isn't directly possible using <code>layout.get</code>. Instead we'll use a bit of python magic to pull the data that we want:

In [10]:
native_preproc_T1 = [f for f in preproc_T1 if f not in mni_preproc_T1]
native_preproc_T1

[<BIDSImageFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-10438/anat/sub-10438_desc-preproc_T1w.nii.gz'>,
 <BIDSImageFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-10788/anat/sub-10788_desc-preproc_T1w.nii.gz'>,
 <BIDSImageFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-11108/anat/sub-11108_desc-preproc_T1w.nii.gz'>,
 <BIDSImageFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-50052/anat/sub-50052_desc-preproc_T1w.nii.gz'>]

Similarily fMRI data can be pulled by specifying <code>datatype='func'</code> and using the <code>desc</code> argument as appropriate:

**Exercise: Pulling Functional Data**

1. Get the list of **all** preprocessed functional data
2. Get the list of functional data in MNI152NLin2009cAsym space
3. Get the list of functional data in T1w space (native)


Note that T1 space fMRI data can be pulled using `space="T1w"` (this is unlike the T1w data which required you to do some filtering)

*All Functional Data*

In [11]:
func_data = layout.get(datatype='func', desc='preproc')
func_data

[<BIDSJSONFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-10438/func/sub-10438_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.json'>,
 <BIDSImageFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-10438/func/sub-10438_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'>,
 <BIDSJSONFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-10788/func/sub-10788_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.json'>,
 <BIDSImageFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-10788/func/sub-10788_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'>,
 <BIDSJSONFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-10788/func/sub-10788_task-rest_space-T1w_desc-preproc_bold.json'>,
 <BIDSImageFile filename='/home/jerry/projects/wor

*MNI152NLin2009cAsym Functional Data*

In [12]:
mni_func_data = layout.get(datatype='func', desc='preproc', space='MNI152NLin2009cAsym')
mni_func_data

[<BIDSJSONFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-10438/func/sub-10438_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.json'>,
 <BIDSImageFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-10438/func/sub-10438_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'>,
 <BIDSJSONFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-10788/func/sub-10788_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.json'>,
 <BIDSImageFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-10788/func/sub-10788_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'>,
 <BIDSJSONFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-11108/func/sub-11108_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.json'>,
 <BIDSImageFile filename='/home/je

*Native Functional Data*

In [13]:
t1w_func_data = layout.get(datatype='func', desc='preproc', space='T1w')
t1w_func_data

[<BIDSJSONFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-10788/func/sub-10788_task-rest_space-T1w_desc-preproc_bold.json'>,
 <BIDSImageFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-10788/func/sub-10788_task-rest_space-T1w_desc-preproc_bold.nii.gz'>,
 <BIDSJSONFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-50052/func/sub-50052_task-rest_space-T1w_desc-preproc_bold.json'>,
 <BIDSImageFile filename='/home/jerry/projects/workshops/SDC-BIDS-fMRI/data/ds000030/derivatives/fmriprep/sub-50052/func/sub-50052_task-rest_space-T1w_desc-preproc_bold.nii.gz'>]

Now that we have a handle on how fMRIPREP preprocessed data is organized and how we can pull this data. Let's start working with the actual data itself!